In [84]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import pickle

In [85]:
with open("./data/details_dict.pkl", "rb") as f:
    details_dict = pickle.load(f)

In [122]:
def parse_comment(comment):
    json = {}

    content = comment.find("span", class_="short")
    if content is not None:
        json["content"] = content.contents[0]

    votes = comment.find("span", class_=["votes", "votes-count"])
    if votes is not None:
        json["votes"] = int(votes.contents[0])

    rating = comment.find("span", class_="rating")
    if rating is not None:
        json["rating"] = rating.get("title")

    return json if len(json) > 0 else None

In [124]:
path = "./data/detail"
count = 0
top_comments = []

for key in details_dict:
    count += 1
    if count % 100 == 0:
        print(f"Progress: {count}")
    with open(os.path.join(path, f"{key}.html"), 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f)
        comments = soup.find("div", id="hot-comments")
        c = comments.find("div", class_="comment-item")
        while c is not None:
            parsed = parse_comment(c)
            if parsed is not None:
                parsed["movie_id"] = int(key)
                parsed["movie_name"] = details_dict[key]["name"]
                top_comments.append(parsed)
            c = c.find_next_sibling()

Progress: 100
Progress: 200
Progress: 300
Progress: 400
Progress: 500
Progress: 600
Progress: 700
Progress: 800
Progress: 900
Progress: 1000
Progress: 1100
Progress: 1200
Progress: 1300
Progress: 1400
Progress: 1500
Progress: 1600
Progress: 1700
Progress: 1800
Progress: 1900
Progress: 2000
Progress: 2100
Progress: 2200
Progress: 2300
Progress: 2400
Progress: 2500
Progress: 2600
Progress: 2700
Progress: 2800
Progress: 2900
Progress: 3000
Progress: 3100


In [125]:
df = pd.json_normalize(top_comments)
df.head()

,content,votes,rating,movie_id,movie_name
0,三星半吧。看之前看到此片被豆瓣一些影评人吹得天花乱坠，但看了之后挺失望的。缺乏味道、情怀和批...,904,还行,10344754,毒战 毒戰
1,真的太惊世骇俗了！真没想到合拍片能拍成这样！杜琪峰仍然是我心中的NO.1！！！除了钟汉良李光...,438,力荐,10344754,毒战 毒戰
2,好想甩出一麻袋星！,370,力荐,10344754,毒战 毒戰
3,整部戏的节奏棒极了，调度和表演同样甩出那些所谓的内地“零差评”电视电影或抗日神剧半个地球，故...,154,推荐,10344754,毒战 毒戰
4,足有五六场以上的戏让鄙人惊叫：好鬼搞嘢~,94,力荐,10344754,毒战 毒戰


In [126]:
len(df)

15920

In [127]:
df.to_csv("./data/top_comments_df.csv")

## Comments with most upvotes?

In [128]:
df.sort_values("votes", ascending=False).head(50)

,content,votes,rating,movie_id,movie_name
7690,王传君所有不被外人理解的坚持，都在这一刻得到了完美释放。他不是关谷神奇，他是王传君。\n你看...,48768,力荐,26752088,我不是药神
10660,面对坠楼，他们忙着拍照发微信，只有她为她盖上衣服。\n面对欺凌，他们假装没看见，只有她选择报...,44054,推荐,30166972,少年的你
14555,白居易和空海分别获得各自朋友圈当日微信运动冠军。,32427,还行,5350027,妖猫传
10440,又名《当王思聪是怎样一种体验》，表面上是喜剧片，其实内在无比黑暗。\n有钱可以让最高傲强势的...,29046,推荐,27605698,西虹市首富
8080,看到海报和预告片，人们第一反应就是：哪吒太丑了。\n觉得丑就对了，因为本片讲的就是打破偏见。...,25639,推荐,26794435,哪吒之魔童降世
14556,一流的画面制作，可惜故事不入流，人物都是提线木偶。我心中的大唐不是这么妖里妖气的。从头到尾都...,25587,还行,5350027,妖猫传
15610,有一种老派的好。真正的青春就该歌颂勇气和爱，该歌颂对自我的寻找，要在人性趋利避害的恶里曲曲折...,25469,力荐,6874741,无问西东
9780,让人印象最深的片段是他们的车在南部的乡间小路上抛锚，和田里劳作的黑奴对视的那一幕，无声却发出巨响。,24168,力荐,27060077,绿皮书 Green Book
3060,“你看，这个孩子好懂事啊。”“你怎么知道她是不是害怕、沉默、妥协呢？”,23514,力荐,25716096,狗十三
10496,唐探真的构成了一个唐探宇宙，可以说是中国内陆第一部有宇宙观的电影了。原来的演员一直都在，而且...,22863,力荐,27619748,唐人街探案3


In [97]:
movies = pd.read_csv("./data/movies_prop_df.csv", index_col=0)
movies.head()

,id,name,year,description,genre,ratingCount,bestRating,worstRating,ratingValue,directorMaxEffect,...,y2014,y2015,y2016,y2017,y2018,y2019,y2020,y2021,y2022,dPropaganda
0,10344754,毒战 毒戰,2012,刚刚结束一次危险任务的津海市缉毒大队队长张雷（孙红雷 饰），在医院意外见到因车祸入院治疗的香...,"['剧情', '动作', '犯罪']",210072,10,2,7.5,34235,...,0,0,0,0,0,0,0,0,0,0
1,10355621,粉红女郎之爱人快跑,2013,为了泡到一个男朋友，结婚狂（阿雅 饰）碰见魔术师高明（锦荣 饰）后，便告诉对方自己怀了他的孩...,"['喜剧', '爱情']",1826,10,2,3.2,65,...,0,0,0,0,0,0,0,0,0,0
2,10355633,绝命藏宝图,2012,一对不食人间烟火的青年男女——彭少文和程雪珂狼狈地逃到余姚县城门口，他们身后有一群凶神恶煞的...,"['动作', '悬疑']",78,10,2,3.2,43,...,0,0,0,0,0,0,0,0,0,0
4,10430281,七个隆咚锵咚锵,2012,韩峰（韩兆 饰）在一天之内丢掉了工作和恋人，一无所有的他陷入了人生的低谷之中。然而，韩峰的运...,"['喜剧', '爱情']",506,10,2,4.9,115,...,0,0,0,0,0,0,0,0,0,0
5,10437802,武当少年 武當少年,2010,主人公翔翔原本是一个天真浪漫，活泼好动的孩子，在一次观看武当武校在台湾的表演后，萌生了赴武当...,"['剧情', '儿童']",68,10,2,6.1,2,...,0,0,0,0,0,0,0,0,0,0


In [129]:
prop_movie_comments = df[df.movie_id.isin(movies[movies.dPropaganda == 1].id)]
prop_movie_comments.head()

,content,votes,rating,movie_id,movie_name
100,金家王朝的洗脑大作,43,很差,10478122,平壤之约 평양에서의 약속
101,在河蟹中完成猎奇，在猎奇中完成装逼，在装逼中完成洗脑，在洗脑中完成自慰，赞！没将军的客串，特...,22,很差,10478122,平壤之约 평양에서의 약속
102,阿里郎思密达！,1,很差,10478122,平壤之约 평양에서의 약속
103,呵呵。,10,很差,10478122,平壤之约 평양에서의 약속
104,竟然有点小吐槽，万幸没有出现金将军,2,较差,10478122,平壤之约 평양에서의 약속


In [130]:
prop_movie_comments.sort_values("votes", ascending=False).head(50)

,content,votes,rating,movie_id,movie_name
5920,开篇长镜头惊险大气引人入胜 结合了水平不俗的快剪下实打实的真刀真枪 让人不禁热血沸腾 特别弹...,21162,推荐,26363254,战狼2
13080,过程才是现实，结尾只是理想。,16550,推荐,35312437,奇迹·笨小孩
11170,关晓彤的戏份真是多余……,14192,推荐,30295905,中国机长
5921,下一部拍喜剧吧，整个片子真感觉挺搞笑的,12969,较差,26363254,战狼2
12625,为什么这些导演觉得旅游村、度假村可以代表中国农村呢？,11269,还行,35051512,我和我的家乡
12855,这个电影可能只拍了一天，从早拍到晚，因为机位多，硬是剪出了两个小时,9876,还行,35155748,金刚川
7725,7.5/10. 怀着9分的期望去看的，遗憾的发现影片远没有我想象中的那么好，虽然本身质量也够...,9049,推荐,26754233,八佰
12626,徐峥的才华确实被低估了。“乡村教师”这个题材的电影展现出他某个程度的细腻——他很擅长从一个小...,8549,力荐,35051512,我和我的家乡
13270,吴京终于以主角光环➕360度慢镜头全方位无死角的方式展示了他的牺牲,8039,还行,35613853,长津湖之水门桥
3610,跟彭于晏谈了十年恋爱结果你还要吸毒。。感觉你脑子不好。。,7896,推荐,25815034,湄公河行动


## Top worst comments?

In [132]:
df[df.rating == "很差"].sort_values("votes", ascending=False).head(10)

,content,votes,rating,movie_id,movie_name
3670,年度最烂，真的是年度最烂，前半部像杨洋和刘亦菲主持的少儿节目，后半部毁天灭地只为衬托他们狗屎...,18792,很差,25823277,三生三世十里桃花
5365,刘亦菲也许不会想到，自己也有演技碾压对手的一天,18017,很差,26290642,致青春·原来你还在这里
4480,我个傻逼花12点到2点的黄金睡眠时间来电影院听了2个小时的鸡汤独白 刚刚看了一刻钟我就打开豆...,13903,很差,25911694,摆渡人
2805,爱情公寓大电影之抢你钱没商量,10775,很差,24852545,爱情公寓
2750,看完盗墓笔记 向九层妖塔道歉,10509,很差,24827387,盗墓笔记
5850,我竟没有想到在吴亦凡的衬托下，唐嫣的演技还是显得那么的烂。,7851,很差,26351812,欧洲攻略
5590,好奇心使我点进去，求生欲使我退出来,7528,很差,26322774,纯洁心灵·逐梦演艺圈
6492,影院出来的过来人劝大家都别过来，看盗版都觉得浪费时间的片。看到这么多人骂我就放心了,7324,很差,26581837,上海堡垒
5885,你们电影院怎么搞的，为什么要放枪版？嗯，什么？你说这就是正版？。。。,7309,很差,26354336,爵迹
5250,拜托快点路过别让我看见。,7001,很差,26280528,从你的全世界路过


In [134]:
prop_movie_comments[prop_movie_comments.rating == "很差"].sort_values("votes", ascending=False).head(20)

,content,votes,rating,movie_id,movie_name
2715,好羞耻，看得我一阵阵脸红,3899,很差,24753810,战狼
10710,世纪悍匪张子强被拍的跟TM憨批一样 我服,1283,很差,30175306,追龙Ⅱ 追龍2：賊王
6115,用的抗日小人书的事迹体，把《心爱的土琵琶》歌词具象化了，将《道游击队》《洪湖赤卫队》《雷玛根...,608,很差,26389069,铁道飞虎
10016,这种政治旋律不能随便评论 但是你们想洗脑底层 拿出一点诚意 就这烂片 怎么洗脑底层？,403,很差,27102739,中国蓝盔
5569,请看看王苹版本的《永不消逝的电波》吧！请看看王苹版本的《永不消逝的电波》吧！请看看王苹版本的...,297,很差,26316956,密战
15225,网友给的71.7%的一星评价 却显示7.0的高分。 伟大的作品 又一次震撼豆瓣网友脆弱的心灵啊,209,很差,6519585,飞天
10245,不尊重历史不还原历史的作品都是垃圾,204,很差,27180974,浴血广昌
5671,视效，剧情史诗级的灾难……ps2时代过场动画水平。改档，下映是因为自己觉着丢人么？,189,很差,26331700,大轰炸
5670,导演一直哭穷卖惨，不如少请几个明星，少做几个爆破特效，多花点时间打磨一下剧本。幸好国内没有上...,182,很差,26331700,大轰炸
11698,周一围的角色，从头到尾都是傻逼一样的存在。编剧还嫌不够乱，又弄出了三小姐和五爷各种傻叉角色来...,166,很差,30454698,解放·终局营救
